In [2]:
import configparser
CONFIG = configparser.ConfigParser()
CONFIG.read('credentials.ini')

['credentials.ini']

In [3]:
import tweepy
import csv 

class StdOutListener(tweepy.Stream):
    def on_status(self, status):
        # saves the tweet object
        user = status.author.screen_name
        location = status.user.location
        timeTweet = status.created_at
        source = status.source
        tweet = status.text
        writer.writerow([tweet,timeTweet,user,location,source])


In [4]:
def start_streaming():
     # Opening a CSV file to save the gathered tweets
        with open("tweets.csv", 'w') as file:
            global writer
            writer = csv.writer(file)

            # Add a header row to the CSV
            writer.writerow(["Tweet", "Date", "User","Location","Source"])

            # Creating a Stream
            try:
                listener = StdOutListener(CONFIG['DEFAULT']['consumer_key'], CONFIG['DEFAULT']['consumer_secret'], CONFIG['DEFAULT']['access_token'], CONFIG['DEFAULT']['access_token_secret'])
                listener.filter(track=["hallmark christmas"])  
            except KeyboardInterrupt:
                #User pressed ctrl+c or cmd+c -- get ready to exit the program
                print("- KeyboardInterrupt caught. Closing stream and exiting.")
                listener.disconnect()
                pass 


In [5]:
start_streaming()

Stream encountered an exception
Traceback (most recent call last):
  File "C:\Users\an111789\Anaconda3\lib\site-packages\tweepy\streaming.py", line 133, in _connect
    self.on_data(line)
  File "C:\Users\an111789\Anaconda3\lib\site-packages\tweepy\streaming.py", line 387, in on_data
    return self.on_status(status)
  File "<ipython-input-3-1f08fde35ab4>", line 12, in on_status
    writer.writerow([tweet,timeTweet,user,location,source])
  File "C:\Users\an111789\Anaconda3\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u2605' in position 29: character maps to <undefined>


In [ ]:
import pandas as pd

search_words = "#squidgame"
date_since = "2021-10-01"
tweets = tweepy.Cursor(api.search_tweets,
              q=search_words, lang="en").items(300)

tweets_sorted = [[tweet.user.screen_name, tweet.geo, tweet.user.location, tweet.text] for tweet in tweets]


In [ ]:
tdf = pd.DataFrame(data=tweets_sorted, columns=['user', 'coordinates','location', 'tweet'])
print(tdf)

In [ ]:
locs = tdf['location'].value_counts()
print(locs)

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent='twitter-analysis-client')
limited = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def find_location(row):
    place = row['location']
    location = limited(place)
    
    if location != None:
        return location.latitude, location.longitude
    else:
        return "Not Found", "Not Found"

tdf[['latitude','longitude']] = tdf.apply(find_location, axis="columns", result_type="expand")
tdf


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 20})
plt.rcParams['figure.figsize'] = (20, 10)

In [ ]:
import cartopy.crs as ccrs
from matplotlib.patches import Circle

lats = tdf['latitude'].dropna()
longs = tdf['longitude'].dropna()

ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
    
for i in range(0, len(lats)):
    ax.plot(longs[i], lats[i], 'ro', transform=ccrs.PlateCarree())
                                                                                              
ax.coastlines(resolution='50m')
ax.set_global()

plt.show()
